In [2]:
import requests
from dotenv import load_dotenv
import os
import numpy as np
load_dotenv()
API_KEY = os.getenv('API_KEY')

In [3]:
sports_url = f'https://api.the-odds-api.com/v4/sports/?apiKey={API_KEY}'

sports_response = requests.get(sports_url)
sports_data = sports_response.json()

sports = []
for sport in sports_data:
    sports.append(sport['key'])

In [7]:
s_list = []
for sport in sports:
    sport_team_odds_url = f'https://api.the-odds-api.com/v4/sports/{sport}/odds/?apiKey={API_KEY}&regions=us&markets=h2h,spreads&oddsFormat=american'
    response = requests.get(sport_team_odds_url)
    data = response.json()
    s_list.append([sport, len(data)])

print(s_list)

[['americanfootball_ncaaf', 21], ['americanfootball_ncaaf_championship_winner', 1], ['americanfootball_nfl_super_bowl_winner', 1], ['aussierules_afl', 9], ['baseball_kbo', 5], ['baseball_mlb', 25], ['baseball_mlb_world_series_winner', 1], ['baseball_ncaa', 3], ['baseball_npb', 6], ['basketball_euroleague', 9], ['basketball_nba', 15], ['basketball_nba_championship_winner', 1], ['basketball_ncaab', 8], ['basketball_ncaab_championship_winner', 1], ['boxing_boxing', 46], ['cricket_international_t20', 1], ['cricket_ipl', 4], ['cricket_test_match', 1], ['golf_masters_tournament_winner', 1], ['golf_pga_championship_winner', 1], ['golf_the_open_championship_winner', 1], ['golf_us_open_winner', 1], ['icehockey_nhl', 16], ['icehockey_nhl_championship_winner', 1], ['icehockey_sweden_allsvenskan', 2], ['icehockey_sweden_hockey_league', 2], ['mma_mixed_martial_arts', 63], ['politics_us_presidential_election_winner', 1], ['rugbyleague_nrl', 8], ['soccer_australia_aleague', 14], ['soccer_austria_bund

In [5]:
for team in data:
    print('\n')
    print(f"{team['bookmakers'][0]['markets'][0]['outcomes'][0]['name']} vs {team['bookmakers'][0]['markets'][0]['outcomes'][1]['name']}")
    
    for odds in team['bookmakers']:
        print(f"{odds['title']}: {odds['markets'][0]['outcomes'][0]['price']}  {odds['markets'][0]['outcomes'][1]['price']}") 
        # print(f"odds['title']: {odds['markets']['outcomes'][0]}")
        
# For negative odds
# profit = bet * (100/odds)

# For positive odds
# profit = bet * (odds/100)

# if 1/profit + 1/profit < 100%, then there is an arbitrage opportunity






Colorado Avalanche vs Columbus Blue Jackets
BetRivers: -335  260
Unibet: -335  260
FanDuel: -350  270
BetOnline.ag: -315  275
LowVig.ag: -315  275
PointsBet (US): -350  260
DraftKings: -360  285
MyBookie.ag: -350  270
BetUS: -350  290
BetMGM: -350  280
Caesars: -350  275
Bovada: -340  270
SuperBook: -340  280
WynnBET: -335  295


Detroit Red Wings vs Tampa Bay Lightning
Unibet: 163  -200
BetRivers: 163  -200
FanDuel: 146  -176
BetOnline.ag: 155  -174
LowVig.ag: 155  -174
DraftKings: 150  -180
PointsBet (US): 160  -190
MyBookie.ag: 165  -200
BetUS: 155  -175
BetMGM: 150  -175
Caesars: 158  -190
Bovada: 153  -177
SuperBook: 158  -178
WynnBET: 158  -175


Florida Panthers vs Toronto Maple Leafs
Unibet: -134  110
BetRivers: -132  110
FanDuel: -142  118
BetOnline.ag: -135  120
LowVig.ag: -135  120
DraftKings: -135  114
PointsBet (US): -130  110
MyBookie.ag: -130  110
BetUS: -135  115
BetMGM: -135  115
Caesars: -135  115
Bovada: -140  120
SuperBook: -130  115
WynnBET: -130  118


New York 

In [28]:
def is_SBA(b1, b2):
    
    if b1 < 0:
        profit1 = 100/abs(b1)
    else:
        profit1 = b1/100
        
    if b2 < 0:
        profit2 = 100/abs(b2)
    else:
        profit2 = b2/100
    
    return ((1/(1 + profit1)) + (1/ (1 + profit2))) 
    
def find_probs(b1, b2):
    
    if b1 < 0:
        p1 = b1 / (b1 - 100)
    else:
        p1 = 100 / (b1 + 100)
        
    if b2 < 0:
        p2 = b2 / (b2 - 100)
    else:
        p2 = 100 / (b2 + 100)
        
    return p1 + p2
    

In [42]:
odds1 = 100
odds2 = 100

print(is_SBA(odds1,odds2))
print(find_probs(odds1,odds2))

1.0
1.0


In [ ]:
##practice hedge odds
money = 100
bucks = -130
pel = -110
